In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split

# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

In [48]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [49]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [90]:
def weighted_random_recommendation(items_weights, n=5):
    
    recs = items_weights.sample(n).sort_values(by='weights', ascending=False)
    
    return np.array(recs['items'])
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    

In [92]:
%%time
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

items, counts = np.unique(data_train.item_id, return_counts=True)
weights = [np.log(el) for el in counts]
items_weights = pd.DataFrame({'items': items, 
                              'weights': weights})

result['random_recommendation'] = [weighted_random_recommendation(items_weights, n=5) for el in result['user_id']]

result.head(2)


CPU times: user 7.69 s, sys: 7.27 ms, total: 7.69 s
Wall time: 7.7 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6442834, 10456856, 9220887, 6463681, 15604203]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1100096, 972418, 1113467, 15781015, 1427860]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [100]:
def popularity_recommendation(data, n=5, n_top = 5000):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular[:n_top].sample(n).item_id
    
    return recs.tolist()

In [101]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 197 ms, sys: 0 ns, total: 197 ms
Wall time: 194 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6442834, 10456856, 9220887, 6463681, 15604203]","[1037451, 855257, 1096626, 1007604, 920109]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1100096, 972418, 1113467, 15781015, 1427860]","[1037451, 855257, 1096626, 1007604, 920109]"


## ItemItemRecommender

In [153]:
#код из урока
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,6666,969544
1,202291,35911
2,397896,1214994
3,420647,168661
4,480014,371107


In [104]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/home/vasiliy/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [107]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)


In [109]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [140]:
%%time
# напишем функцию принимающую K - кол-во билжайших соседей и далее строим,обучаем модель. возврашает функция пресижен при определенном К
def ItemItem(k):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей


    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)


    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)

    result['itemitem'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                                  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                    user_items=sparse_user_item,   # на вход user-item matrix
                                                                    N=5, 
                                                                    filter_already_liked_items=False, 
                                                                    filter_items=None, 
                                                                    recalculate_user=True)
                                                                                 ])

    err = round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(),4)
    return err

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 16 µs


In [144]:
# эксперементируем с разными параметрами
k = [el for el in range(1,20)]
err = [ItemItem(el) for el in k]


  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

...

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [145]:
pd.DataFrame({'k': k, 
              'pres': err})

,k,pres
0,1,0.1923
1,2,0.1920
2,3,0.1861
3,4,0.1449
4,5,0.1368
5,6,0.1421
6,7,0.1450
7,8,0.1472
8,9,0.1485
9,10,0.1509


можно сделать вывод что наилучшая метрика будет при К = 1 

In [128]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0004:random_recommendation
0.0032:popular_recommendation
0.1368:itemitem
